In [ ]:
%matplotlib inline

In [ ]:
import functools
from typing import Tuple
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import linalg

# PHYS 395 - week 5

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (10, 7)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Vector/matrix operations

## Addition/subtraction

Note that the `+` operation when used on Python lists concatenates lists. However, for NumPy arrays, the `+` operation does elementwise addition.

In [ ]:
a_list = [1.1, -2.1, 0.0]
b_list = [2.0, 1.1, -0.5]

a = np.array(a_list)
b = np.array(b_list)

# + for lists
print("for lists: a + b = %s" % (a_list + b_list))

# + for NumPy arrays
print("for arrays: a + b = %s" % (a + b))

We can also do `-` for elementwise subtraction.

In [ ]:
print("a - b = %s" % (a - b))

## Multiplication by a scalar

Using NumPy arrays we can also do elementwise scalar multiplication. (Note that the `*` operator does not work between a float and a Python list.)

In [ ]:
v = np.array([1.0, -2.1, 3.0])

print(2.0 * v)

## Vector/matrix products

Using the `*` operator will perform elementwise multiplication (the $i$th element of the first array is multiplied by the $i$th element of the second array).

In [ ]:
print(a * b)

### Dot product

There are a few different ways of taking the dot product.

In [ ]:
# Using np.dot
print(np.dot(a, b))

# Using *
print(np.sum(a * b))

Using NumPy's `dot` function (or the `@` operator) allows us to calculate proper matrix products.

In [ ]:
sigma_x = np.array([[0, 1], [1, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

Let's calculate $\sigma_x \sigma_z$.

In [ ]:
print(sigma_x @ sigma_z)

And $\sigma_z \sigma_x \sigma_z$.

In [ ]:
print(sigma_z @ sigma_x @ sigma_z)

### Vector norms

Calculating Euclidean norms is simple with NumPy. Here we'll calculate $| a |$.

In [ ]:
print(np.linalg.norm(a))

Or we determine the $|a|$ using the dot product.

In [ ]:
print(np.sqrt(np.dot(a, a)))

### Cross product

We can also easily compute cross products. Let's calculate $a \times b$.

In [ ]:
print(np.cross(a, b))

And let's verify this by evaluating the cross product ourselves:

\begin{align*}
    a \times b
        &= (a_y b_z - a_z b_y) \hat{i}
            + (a_z b_x - a_x b_z) \hat{j}
            + (a_x b_y - a_y b_x) \hat{z}
            \\
        &= ((-2.1) (-0.5) - (0) (1.1)) \hat{i}
            + ((0) (1.1) - (1.1) (-0.5)) \hat{j}
            + ((1.1) (1.1) - (-2.1) (2.0)) \hat{z}
            \\
        &= 1.05 \hat{i}
            + 0.55 \hat{j}
            + 5.41 \hat{z}
            ,
\end{align*}

which agrees with the computed value.

# Solving systems of equations

## LU decomposition

First let's create a random 5 x 5 matrix $A$ and length 5 array $b$. We'll explore how we can solve $A x = b$.

In [ ]:
a = np.random.rand(5, 5)
b = np.random.rand(5)

print("A =\n%s\n\nb = %s" % (a, b))

Let's try using SciPy's `solve` function to solve for $x$.

In [ ]:
x = linalg.solve(a, b)

print("x = %s" % x)

Let's verify the solution. Note that we don't want to use `==` here to test for equality since generally there will be some negligible error.

In [ ]:
print(np.allclose(a @ x, b))

Now we'll explicitly carry out an LU decomposition.

In [ ]:
p, l, u = linalg.lu(a)

print("P =\n%s\n\nL =\n%s\n\nU =\n%s" % (p, l, u))

Let's verify that $P L U = A$.

In [ ]:
print(np.allclose(p @ l @ u, a))

Now we'll solve two equations. The first is $L y = P^{-1} b$ for $y$, then $U x = y$ for $x$. Note that $P^{-1} = P^T$ since $P$ is a permutation matrix; this is a straightforward result from linear algebra.

In [ ]:
y_lu = linalg.solve(l, p.transpose() @ b)
x_lu = linalg.solve(u, y_lu)

print("x = %s" % x_lu)

Note that this agrees with our earlier calculation.

## System of linear equations problems

### Resistor chain circuit

Here we need to consider the resistor chain circuit shown in the lab script.

Here we have that $V_0$ is given. For voltages $V_i$ with $i = 2, \ldots, N - 1$, Kirchoff's current law gives the result

\begin{equation}
    I_{i - 2, i} + I_{i - 1, i} + I_{i + 1, i} + I_{i + 2, i} = 0
    ,
\end{equation}

where $I_{j, i}$ is the current flowing from node $j$ to node $i$. Note that we have taken the $N + 1$th node to be the ground.

Applying Ohm's law and multiplying through by the resistance $R$ (which is the same for all resistors) we have

\begin{align}
    &\frac{1}{R} \left(\Delta V_{i - 2, i} + \Delta V_{i - 1, i} + \Delta V_{i + 1, i} + \Delta V_{i + 2, i} \right) = 0 \\
    &\Rightarrow V_{i - 2} + V_{i - 1} + V_{i + 1} + V_{i + 2} - 4 V_i = 0
    .
\end{align}

For the case of $i = 1$ we have

\begin{align}
    &I_{0, 1} + I_{2, 1} + I_{3, 1} = 0 \\
    &\Rightarrow V_0 + V_2 + V_3 - 3 V_1 = 0
    ;
\end{align}

and for $i = N$,

\begin{align}
    &I_{N - 2, N} + I_{N - 1, N} + I_{*, N} = 0 \\
    &\Rightarrow V_{N - 2} + V_{N - 1} - 3 V_N = 0
    .
\end{align}

Here, $*$ denotes the ground node.

Now let's set up a function that gives us $A$ and $b$ so that we can solve these equations.

In [ ]:
def resistor_chain_matrix(n: int, v0: float) -> Tuple[np.ndarray, np.ndarray]:
    """Returns the resistor chain matrices A and b"""
    # Construct A first
    A = 4 * np.eye(n)

    # Add off diagonals
    A = functools.reduce(lambda mat, k: mat - np.eye(n, k=k), [-2, -1, 1, 2], A)

    # Adjust the non-symmetric entries
    A[0][0] = 3
    A[n - 1, -1] = 3

    b = np.zeros(n)
    b[0] = v0
    b[1] = v0

    return (A, b)

We will test this for the $N = 6$ case where $V_0 = 4$ volts.

In [ ]:
x = linalg.solve(*resistor_chain_matrix(6, 4))

print("x = %s" % x)

Now we'll use $N = 10000$. For efficiency reasons, we want to use a "banded representation" of our matrix $A$.

In [ ]:
n = 10 ** 4
v0 = 4

# Construct banded representation of A
a = - 1 * np.ones((5, n))
a[2] = 4 * np.ones(n)
a[2][0] = 3
a[2][-1] = 3

# Get b
b = np.zeros(n)
b[0] = v0
b[1] = v0

In [ ]:
x = linalg.solve_banded((2, 2), a , b)

Now let's plot this solution.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot data
plt.plot(np.arange(1, n + 1), x)

# Labels
ax.set_xlabel(r"$i$")
ax.set_ylabel(r"$V_i$");

This result isn't unreasonable, but was unexpected for me that it appears to be perfectly linear.

### Resistor capacitor circuit

Now we need to solve for the voltages in a circuit involving capacitors.